In [1]:
# Dependencies and Setup
import csv 
from config import api_key
import pandas as pd
import numpy as np
import requests
import time
import json
import pymongo
from sqlalchemy import create_engine
import datetime as dt
url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"
units = "metric"

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
csv_file = 'Resources/2018_airline_delay_causes.csv'
airline_data_df = pd.read_csv(csv_file, delimiter = ',')
airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,3.0,1.00,0.06,...,0.0,0.00,0.0,0.0,104.0,54.0,1.0,49.0,0.0,0.0
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,21.0,7.17,1.16,...,0.0,5.92,3.0,0.0,897.0,344.0,37.0,226.0,0.0,290.0
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,8.0,0.22,0.35,...,0.0,1.82,0.0,0.0,353.0,9.0,18.0,233.0,0.0,93.0
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,11.0,1.75,1.08,...,0.0,5.19,3.0,0.0,657.0,83.0,34.0,130.0,0.0,410.0
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,12.0,0.82,3.00,...,0.0,1.55,0.0,0.0,484.0,27.0,136.0,207.0,0.0,114.0


In [4]:
airline_data_df.columns

Index(['year', ' month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', ' weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       ' arr_delay', ' carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [5]:
new_airline_data_df = airline_data_df[['year',' month','carrier','carrier_name','airport','airport_name','arr_flights','arr_cancelled', ' arr_delay', ' carrier_delay','weather_delay']].copy()

new_airline_data_df.head(10)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0
5,2018,1,MQ,Envoy Air,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",112.0,3.0,2198.0,567.0,101.0
6,2018,1,MQ,Envoy Air,BZN,"Bozeman, MT: Bozeman Yellowstone International",36.0,0.0,227.0,43.0,27.0
7,2018,1,MQ,Envoy Air,CAE,"Columbia, SC: Columbia Metropolitan",64.0,1.0,611.0,120.0,150.0
8,2018,1,MQ,Envoy Air,CHO,"Charlottesville, VA: Charlottesville Albemarle",53.0,3.0,696.0,123.0,6.0
9,2018,1,MQ,Envoy Air,CHS,"Charleston, SC: Charleston AFB/International",63.0,12.0,291.0,108.0,1.0


In [6]:
new_airline_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20231 entries, 0 to 20230
Data columns (total 11 columns):
year              20231 non-null int64
 month            20231 non-null int64
carrier           20231 non-null object
carrier_name      20231 non-null object
airport           20231 non-null object
airport_name      20231 non-null object
arr_flights       20214 non-null float64
arr_cancelled     20214 non-null float64
 arr_delay        20214 non-null float64
 carrier_delay    20214 non-null float64
weather_delay     20214 non-null float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.7+ MB


In [7]:
new_airline_data_df[['City','State/Airport']] = new_airline_data_df.airport_name.str.split(expand=True,pat= ':')

In [8]:
new_airline_data_df[['City','State']] = new_airline_data_df['City'].str.split(expand=True,pat= ',')
new_airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,City,State/Airport,State
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0,Bismarck/Mandan,Bismarck Municipal,ND
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0,Nashville,Nashville International,TN
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0,Boise,Boise Air Terminal,ID
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0,Beaumont/Port Arthur,Jack Brooks Regional,TX
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0,Buffalo,Buffalo Niagara International,NY


In [9]:
#new_airline_data_df[['City','County']] = new_airline_data_df['City'].str.split(expand=True,pat= '/')
#new_airline_data_df.head()

In [10]:
final_airline_df = new_airline_data_df.drop(['airport_name','State/Airport'], axis=1)
final_airline_df.head()

,year,month,carrier,carrier_name,airport,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,City,State
0,2018,1,MQ,Envoy Air,BIS,5.0,0.0,104.0,54.0,1.0,Bismarck/Mandan,ND
1,2018,1,MQ,Envoy Air,BNA,110.0,3.0,897.0,344.0,37.0,Nashville,TN
2,2018,1,MQ,Envoy Air,BOI,32.0,0.0,353.0,9.0,18.0,Boise,ID
3,2018,1,MQ,Envoy Air,BPT,63.0,3.0,657.0,83.0,34.0,Beaumont/Port Arthur,TX
4,2018,1,MQ,Envoy Air,BUF,31.0,0.0,484.0,27.0,136.0,Buffalo,NY


In [11]:
popular_airline = ['LAX','ATL','ORD','DEN','DFW']
top_airline = final_airline_df.loc[final_airline_df['airport'].isin(popular_airline)]
top_airline.head(10)

,year,month,carrier,carrier_name,airport,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,City,State
23,2018,1,MQ,Envoy Air,DFW,4491.0,152.0,48316.0,11983.0,7471.0,Dallas/Fort Worth,TX
81,2018,1,MQ,Envoy Air,ORD,4704.0,166.0,69063.0,18761.0,5530.0,Chicago,IL
114,2018,1,NK,Spirit Air Lines,ATL,629.0,10.0,6276.0,2009.0,135.0,Atlanta,GA
121,2018,1,NK,Spirit Air Lines,DEN,258.0,2.0,3064.0,441.0,120.0,Denver,CO
122,2018,1,NK,Spirit Air Lines,DFW,608.0,6.0,5490.0,2085.0,15.0,Dallas/Fort Worth,TX
129,2018,1,NK,Spirit Air Lines,LAX,682.0,7.0,4683.0,1920.0,84.0,Los Angeles,CA
138,2018,1,NK,Spirit Air Lines,ORD,774.0,10.0,6505.0,964.0,123.0,Chicago,IL
154,2018,1,OH,PSA Airlines Inc.,ATL,68.0,3.0,470.0,228.0,0.0,Atlanta,GA
219,2018,1,OH,PSA Airlines Inc.,ORD,181.0,5.0,4522.0,1498.0,33.0,Chicago,IL
255,2018,1,OO,SkyWest Airlines Inc.,ATL,2115.0,63.0,62234.0,17773.0,4214.0,Atlanta,GA


In [12]:
final_df = top_airline.sort_values(' month')
final_df.head(10)
final_df.to_csv('Resources/final_airline_data.csv')

In [13]:
year = top_airline.groupby(['year',' month','City'], as_index=False).mean()
year.head(10)

,year,month,City,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay
0,2018,1,Atlanta,2041.266667,59.666667,27177.866667,7903.066667,2487.333333
1,2018,1,Chicago,1833.714286,45.714286,27676.500000,7900.714286,1343.142857
2,2018,1,Dallas/Fort Worth,1719.769231,43.000000,16234.846154,5884.000000,1417.076923
3,2018,1,Denver,1402.153846,18.153846,12630.000000,4188.076923,717.153846
4,2018,1,Los Angeles,1482.750000,19.000000,12789.250000,4954.916667,789.833333
5,2018,2,Atlanta,1875.066667,8.933333,17566.533333,6407.000000,951.066667
6,2018,2,Chicago,1668.285714,80.285714,26393.071429,5921.142857,1678.357143
7,2018,2,Dallas/Fort Worth,1574.230769,42.923077,26148.846154,7056.307692,1881.538462
8,2018,2,Denver,1249.846154,10.692308,14111.153846,5019.230769,596.153846
9,2018,2,Los Angeles,1341.000000,10.916667,13593.166667,4881.500000,474.416667


In [14]:
def month_json_to_df(city_name):
    start_date=["2018-01-01","2018-02-01","2018-03-01","2018-04-01","2018-05-01","2018-06-01","2018-07-01","2018-08-01","2018-09-01","2018-10-01","2018-11-01","2018-12-01"]
    end_date=["2018-01-31","2018-02-28","2018-03-31","2018-04-30","2018-05-31","2018-06-30","2018-07-31","2018-08-31","2018-09-30","2018-10-31","2018-11-30","2018-12-31"]
    for i in range(12):
        query_url = f"{url}?key={api_key}&q={city_name}&format=json&extra=2018-01-01&date={start_date[i]}&enddate={end_date[i]}"
        response=requests.get(query_url)
        json_file = response.json()
        city = json_file['data']['request'][0]['query']
    
        date=[]
        weather_desc=[]
        for weather_row in json_file['data']['weather']:
            date.append(weather_row['date'])
            weather_desc.append(weather_row['hourly'][0]['weatherDesc'][0]['value'])
        if i ==0:
            final_DF=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
        else:
            temp_df=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
            final_DF=pd.concat([final_DF, temp_df])
    return final_DF

In [15]:
lax_df=month_json_to_df('Los Angeles')
lax_df.to_csv('Resources/lax_df.csv')
#lax_df = lax_df.rename(columns= {"city":"city_name", "date":"date_of","weather":"weather_desc"})
lax_df.head()

,city,date,weather
0,"Los Angeles, United States of America",2018-01-01,Clear
1,"Los Angeles, United States of America",2018-01-02,Clear
2,"Los Angeles, United States of America",2018-01-03,Clear
3,"Los Angeles, United States of America",2018-01-04,Clear
4,"Los Angeles, United States of America",2018-01-05,Clear


In [16]:
chi_df=month_json_to_df("Chicago")
chi_df.head()

,city,date,weather
0,"Chicago, United States of America",2018-01-01,Clear
1,"Chicago, United States of America",2018-01-02,Clear
2,"Chicago, United States of America",2018-01-03,Clear
3,"Chicago, United States of America",2018-01-04,Partly cloudy
4,"Chicago, United States of America",2018-01-05,Clear


In [17]:
dfw_df=month_json_to_df("Dallas")
dfw_df.head()

,city,date,weather
0,"Dallas, United States of America",2018-01-01,Partly cloudy
1,"Dallas, United States of America",2018-01-02,Clear
2,"Dallas, United States of America",2018-01-03,Clear
3,"Dallas, United States of America",2018-01-04,Clear
4,"Dallas, United States of America",2018-01-05,Overcast


In [18]:
atl_df=month_json_to_df("Atlanta")
atl_df.head()

,city,date,weather
0,"Atlanta, United States of America",2018-01-01,Partly cloudy
1,"Atlanta, United States of America",2018-01-02,Clear
2,"Atlanta, United States of America",2018-01-03,Overcast
3,"Atlanta, United States of America",2018-01-04,Clear
4,"Atlanta, United States of America",2018-01-05,Clear


In [19]:
den_df=month_json_to_df("Denver")
den_df.head()

,city,date,weather
0,"Denver, United States of America",2018-01-01,Clear
1,"Denver, United States of America",2018-01-02,Clear
2,"Denver, United States of America",2018-01-03,Clear
3,"Denver, United States of America",2018-01-04,Clear
4,"Denver, United States of America",2018-01-05,Clear


In [20]:
#den_df.columns

In [21]:
final = pd.concat([lax_df, dfw_df, chi_df, atl_df, den_df])
final_weather = final


In [22]:
final_weather.weather.unique()

array(['Clear', 'Cloudy', 'Partly cloudy', 'Moderate rain',
       'Moderate rain at times', 'Light drizzle', 'Overcast',
       'Light rain', 'Light rain shower', 'Moderate or heavy rain shower',
       'Patchy rain possible', 'Patchy light drizzle', 'Mist',
       'Thundery outbreaks possible', 'Fog', 'Freezing fog'], dtype=object)

In [23]:
final['weather_val']=final['weather'].replace({'Clear':15, 'Overcast':14,'Partly cloudy':13, 'Cloudy':12, 'Fog':11,\
                          'Freezing fog':10, 'Mist':9, 'Patchy light drizzle':8, 'Light drizzle':7, 'Patchy rain possible':6,\
                         'Light rain':5, 'Light rain shower':4, 'Moderate rain at times':3,'Moderate rain':2, 'Moderate or heavy rain shower':1,\
                         'Thundery outbreaks possible':0})


In [24]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 4 columns):
city           1825 non-null object
date           1825 non-null object
weather        1825 non-null object
weather_val    1825 non-null int64
dtypes: int64(1), object(3)
memory usage: 71.3+ KB


In [25]:
final['date']=pd.to_datetime(final['date'])

In [26]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 4 columns):
city           1825 non-null object
date           1825 non-null datetime64[ns]
weather        1825 non-null object
weather_val    1825 non-null int64
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 71.3+ KB


In [27]:
final['month']= final['date'].dt.month

In [28]:
final['year']= final['date'].dt.year

In [29]:
final.head()

,city,date,weather,weather_val,month,year
0,"Los Angeles, United States of America",2018-01-01,Clear,15,1,2018
1,"Los Angeles, United States of America",2018-01-02,Clear,15,1,2018
2,"Los Angeles, United States of America",2018-01-03,Clear,15,1,2018
3,"Los Angeles, United States of America",2018-01-04,Clear,15,1,2018
4,"Los Angeles, United States of America",2018-01-05,Clear,15,1,2018


In [30]:
final.columns

Index(['city', 'date', 'weather', 'weather_val', 'month', 'year'], dtype='object')

In [31]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 6 columns):
city           1825 non-null object
date           1825 non-null datetime64[ns]
weather        1825 non-null object
weather_val    1825 non-null int64
month          1825 non-null int64
year           1825 non-null int64
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 99.8+ KB


In [32]:
final[['city','country']] = final.city.str.split(expand= True,pat=',')

In [33]:
final.head()

,city,date,weather,weather_val,month,year,country
0,Los Angeles,2018-01-01,Clear,15,1,2018,United States of America
1,Los Angeles,2018-01-02,Clear,15,1,2018,United States of America
2,Los Angeles,2018-01-03,Clear,15,1,2018,United States of America
3,Los Angeles,2018-01-04,Clear,15,1,2018,United States of America
4,Los Angeles,2018-01-05,Clear,15,1,2018,United States of America


In [34]:
final_df = final.groupby(['year','month','city'],as_index=False).mean()#.agg(lambda x:x.value_counts().index[0])

final_df.weather_val = np.round(final_df.weather_val)

In [35]:
final_df

,year,month,city,weather_val
0,2018,1,Atlanta,14.0
1,2018,1,Chicago,14.0
2,2018,1,Dallas,14.0
3,2018,1,Denver,15.0
4,2018,1,Los Angeles,14.0
5,2018,2,Atlanta,13.0
6,2018,2,Chicago,13.0
7,2018,2,Dallas,13.0
8,2018,2,Denver,14.0
9,2018,2,Los Angeles,14.0


In [36]:
final_df[final_df.city=='Los Angeles']

,year,month,city,weather_val
4,2018,1,Los Angeles,14.0
9,2018,2,Los Angeles,14.0
14,2018,3,Los Angeles,12.0
19,2018,4,Los Angeles,15.0
24,2018,5,Los Angeles,14.0
29,2018,6,Los Angeles,15.0
34,2018,7,Los Angeles,15.0
39,2018,8,Los Angeles,15.0
44,2018,9,Los Angeles,15.0
49,2018,10,Los Angeles,14.0


In [37]:
final_df.weather_val.unique()

array([14., 15., 13., 12.])

In [38]:
final_df.weather_val.value_counts()

14.0    39
15.0    14
13.0     6
12.0     1
Name: weather_val, dtype: int64

In [39]:
city_list = final.city.unique()

In [40]:
for city in city_list:
    print(city)
#     print(final[(final.city==city)&(final.month == 5)]['weather'].value_counts())
    print(final[(final.city==city)]['weather'].value_counts())

    print('-----------------------')

Los Angeles
Clear                            291
Partly cloudy                     41
Cloudy                            16
Light rain                         4
Overcast                           3
Patchy rain possible               3
Light rain shower                  2
Moderate rain at times             1
Moderate rain                      1
Light drizzle                      1
Moderate or heavy rain shower      1
Patchy light drizzle               1
Name: weather, dtype: int64
-----------------------
Dallas
Clear                          215
Partly cloudy                   64
Overcast                        47
Cloudy                          27
Thundery outbreaks possible      9
Mist                             3
Name: weather, dtype: int64
-----------------------
Chicago
Clear                          171
Overcast                        84
Partly cloudy                   65
Cloudy                          28
Mist                             6
Thundery outbreaks possible      6
Fog  